In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import OneHotEncoder, LabelEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

In [3]:
pref = pd.read_csv('D:/Study/Education/Projects/Restaurant Rating/data/consumer_preferences.csv')
consumer = pd.read_csv('D:/Study/Education/Projects/Restaurant Rating/data/consumers.csv')
cuisine  = pd.read_csv('D:/Study/Education/Projects/Restaurant Rating/data/restaurant_cuisines.csv')
restaurant = pd.read_csv('D:/Study/Education/Projects/Restaurant Rating/data/restaurants.csv')
rating = pd.read_csv('D:/Study/Education/Projects/Restaurant Rating/data/ratings.csv')

In [4]:
consumer = consumer.merge(pref, on = 'Consumer_ID', how = 'left')
consumer = consumer.drop(['Country', 'Latitude', 'Longitude', 'State'], axis = 1)

restaurant = restaurant.merge(cuisine, on = 'Restaurant_ID', how = 'left')
restaurant = restaurant.drop(['State', 'Country', 'Zip_Code', 'Latitude', 'Longitude'], axis =1 )

restaurant['Alcohol_Service'] = restaurant['Alcohol_Service'].replace(np.nan, 'No')
restaurant['Parking'] = restaurant['Parking'].replace(np.nan, 'Limited')
restaurant['Cuisine'] = restaurant['Cuisine'].replace(np.nan, 'Free')

rating_agg = rating.groupby('Consumer_ID').agg({'Overall_Rating':'mean', 'Food_Rating': 'mean', 'Service_Rating':'mean'}).reset_index()

consumer = consumer.merge(rating_agg, how = 'left', on = 'Consumer_ID')
consumer = consumer.drop_duplicates(subset = ['Consumer_ID'],keep = 'first')

rating_res = rating.groupby('Restaurant_ID').agg({'Overall_Rating':'mean', 'Food_Rating':'mean', 'Service_Rating':'mean'})
restaurant = restaurant.merge(rating_res, how = 'left', on = 'Restaurant_ID')

In [5]:
consumer.head()

,Consumer_ID,City,Smoker,Drink_Level,Transportation_Method,Marital_Status,Children,Age,Occupation,Budget,Preferred_Cuisine,Overall_Rating,Food_Rating,Service_Rating
0,U1001,San Luis Potosi,No,Abstemious,On Foot,Single,Independent,23,Student,Medium,American,1.111111,1.222222,1.222222
1,U1002,San Luis Potosi,No,Abstemious,Public,Single,Independent,22,Student,Low,Mexican,1.400000,1.400000,1.000000
2,U1003,San Luis Potosi,No,Social Drinker,Public,Single,Independent,23,Student,Low,Mexican,1.615385,1.692308,1.461538
3,U1004,Cuernavaca,No,Abstemious,Public,Single,Independent,72,Employed,Medium,Bakery,1.875000,1.875000,1.750000
12,U1005,San Luis Potosi,No,Abstemious,Public,Single,Independent,20,Student,Medium,American,1.333333,1.444444,1.000000


In [6]:
restaurant.head()

,Restaurant_ID,Name,City,Alcohol_Service,Smoking_Allowed,Price,Franchise,Area,Parking,Cuisine,Overall_Rating,Food_Rating,Service_Rating
0,132560,Puesto de Gorditas,Ciudad Victoria,No,Yes,Low,No,Open,Public,Regional,0.50,1.00,0.250000
1,132561,Cafe Ambar,Ciudad Victoria,No,No,Low,No,Closed,Limited,Free,0.75,1.00,1.000000
2,132564,Church's,Ciudad Victoria,No,No,Low,No,Closed,Limited,Free,1.25,1.25,1.500000
3,132572,Cafe Chaires,San Luis Potosi,No,No,Low,No,Closed,Yes,Cafeteria,1.00,1.00,0.933333
4,132583,McDonalds Centro,Cuernavaca,No,No,Low,Yes,Closed,Limited,American,1.00,1.00,1.250000


In [7]:
X = restaurant.iloc[:, 2:-3]
y = restaurant.iloc[:, -3]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [8]:
restaurant.to_csv('model.csv')